In [1]:
import os
import yaml

In [2]:
notebook_dir = os.getcwd()
top_dir = os.path.dirname(notebook_dir)
test_dir = os.path.join(top_dir, 'tests')


molecules_dir = os.path.join(test_dir, 'molecules')
water_xyz_file = os.path.join(molecules_dir, 'water.xyz')

yaml_test_file = os.path.join(test_dir, 'test_config.yaml')

In [3]:
with open(water_xyz_file, 'r') as infile:
    geometry = infile.read()
print(geometry)

3
Water
O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [4]:
with open(yaml_test_file, "r") as infile_yaml:
    try:
        input_dict = yaml.safe_load(infile_yaml)
    except yaml.YAMLError as exc:
        print(exc)
input_dict = input_dict['nbed']
input_dict

{'geometry': 'tests/molecules/water.xyz',
 'active_atoms': 2,
 'convergence': 1e-06,
 'qubits': 8,
 'basis': 'STO-3G',
 'xc_functional': 'b3lyp',
 'output': 'openfermion',
 'localisation': 'spade',
 'savefile': 'data/savefile.json'}

In [5]:
geometry = water_xyz_file

In [6]:
from nbed.embed import nbed_driver

n_active_atoms = input_dict['active_atoms']
basis = input_dict['basis']
xc_functional = input_dict['xc_functional']
convergence = input_dict['convergence']
charge = 0
mu_level_shift=1e6
run_ccsd = True
run_fci = True
max_ram = 8_000


loc_method = 'pipekmezey'#input_dict['localisation'] # "spade" #"spade" #"pipekmezey" "ibo" "boys"
run_mu_shift = True
run_huzinaga = True


exp_obj = nbed_driver(
    geometry ,
    n_active_atoms,
    basis,
    xc_functional,
    convergence,
    localization_method= loc_method,
    run_mu_shift = run_mu_shift,
    run_huzinaga = run_huzinaga,
    mu_level_shift = mu_level_shift,
    run_ccsd_emb = run_ccsd,
    max_ram_memory = max_ram,
    run_fci_emb=run_fci,
    perform_global_fci=True,
    get_full_system_H= True)


/Users/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


global FCI: -75.01535247594687


In [7]:
exp_obj.Generate_embedded_H()

orbs: [-18.83525318  -0.93132671  -0.43263415  -0.23435814  -0.14179089
   0.35582625   0.46190031]
act: -76.71295826885648
e_env: -24.4571531839688
two_e_cross: 16.7639428904959
energy_nuc: 9.128283728282938
embedded HF energy MU_SHIFT: -50.682737340673995, converged: True
CCSD Energy MU shift:
	%s -74.63288047544528
FCI Energy MU shift:
	%s -75.60464690817072
embedded HF energy HUZINAGA: -50.68273652091287, converged: True
object test : -50.68273654519365
CCSD Energy HUZ:
	%s -74.63387776610521
FCI Energy HUZ:
	%s -75.26474762053091
number of e- in embedded system: 6
[0 3 4]
[1 2]


In [8]:
exp_obj.print_summary()

********************************************************************************
                         Summary of Embedded Calculation                        
********************************************************************************
global (cheap) DFT calculation -75.27788483404673
********************************************************************************
  Huzinaga calculation
Total energy - active system at RHF level: -74.62417366287576
Total energy - active system at CCSD level: -74.63387776610521
Total energy - active system at FCI level: -75.26474762053091
length of huzinaga embedded fermionic Hamiltonian: 811
number of qubits required: 10
********************************************************************************
  Mu shift calculation
Total energy - active system at RHF level: -74.62417448269424
Total energy - active system at CCSD level: -74.63288047544528
Total energy - active system at FCI level: -75.60464690817072
length of mu embedded fermionic Hamilton

In [9]:
from nbed.embed import get_qubit_hamiltonian

qubit_H_mu = get_qubit_hamiltonian(exp_obj.molecular_ham_MU, 'jordan_wigner')
qubit_H_huz = get_qubit_hamiltonian(exp_obj.molecular_ham_HUZ, 'jordan_wigner')

qubit_H_full_system = get_qubit_hamiltonian(exp_obj.full_system_molecular_ham, 'jordan_wigner')

In [10]:
len(qubit_H_mu.terms)

276

In [11]:
from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [12]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [13]:
import numpy as np
mu_full_quantum = eig_vals_emb_mu[0] 
huz_full_quantum = eig_vals_emb_huz[0]

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

-75.60464690837318
-75.26474762053124
0.3398992878419449


In [14]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

6

In [15]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz[:,0].real)==max(np.abs(eig_vecs_emb_huz[:,0].real)))[0][0]
                                    ).count('1')
N_elec_qubit_state

6